In [18]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LlamaForCausalLM, LlamaTokenizer
from huggingface_hub import login
import os
import torch
import transformers

In [2]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
model_name_or_path = "/data/users/zhangjunlei/tyx/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-hf/snapshots/6fdf2e60f86ff2481f2241aaee459f85b5b0bbb9" # "meta-llama/Llama-2-7b-hf"
gpu_id = 5

In [3]:
device = f"cuda:{gpu_id}"

In [4]:
# tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = LlamaTokenizer.from_pretrained(model_name_or_path)

In [5]:
print(tokenizer)
print(tokenizer.special_tokens_map)

LlamaTokenizer(name_or_path='/data/users/zhangjunlei/tyx/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-hf/snapshots/6fdf2e60f86ff2481f2241aaee459f85b5b0bbb9', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False)}, clean_up_tokenization_spaces=False)
{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}


In [6]:
# no default pad token for llama!
# here we add all special tokens again, because the default ones are not in the special_tokens_map

num_added_tokens = tokenizer.add_special_tokens(
    {
        "bos_token": "<s>",
        "eos_token": "</s>",
        "unk_token": "<unk>",
        "pad_token": "<pad>",
    }
)
assert num_added_tokens in [
    0,
    1,
], "LlamaTokenizer should only add one special token - the pad_token, or no tokens if pad token present."


In [7]:
print(tokenizer)
print(tokenizer.special_tokens_map)

LlamaTokenizer(name_or_path='/data/users/zhangjunlei/tyx/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-hf/snapshots/6fdf2e60f86ff2481f2241aaee459f85b5b0bbb9', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False)
{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}


In [8]:
model = LlamaForCausalLM.from_pretrained(model_name_or_path, low_cpu_mem_usage=True)
model.to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [9]:
print(model)
print(model.config)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [10]:
embedding_size = model.get_input_embeddings().weight.shape[0]
if len(tokenizer) > embedding_size:
    model.resize_token_embeddings(len(tokenizer))

In [11]:
problem = "A $90^\\circ$ rotation around the origin in the counter-clockwise direction is applied to $7 + 2i.$  What is the resulting complex number?"
steps = [
    "I know that a $90^\\circ$ rotation around the origin in the complex plane can be achieved by multiplying the complex number by $i$, since $i$ has a magnitude of 1 and an argument of $90^\\circ$.",
    "So, I can write the rotation as $(7 + 2i)i = 7i + 2i^2.$",
    "To simplify this expression, I recall that $i^2 = -1$, so I can substitute that and get $7i - 2.$",
    "This is the resulting complex number after the rotation.\n\n# Answer\n\n7i - 2",
]

In [12]:
print(model.generation_config)

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.9,
  "transformers_version": "4.31.0"
}



In [30]:
rating2word = {1: "positive", -1: "negative", 0: "neutral"}
rating_words = list(rating2word.values())
rating_token_ids = tokenizer(rating_words, add_special_tokens=False).input_ids
print(rating_token_ids)



# constraint = transformers.Constraint(type="must_include", token="apple")

[[6374], [8178], [21104]]


In [37]:
inputs = tokenizer(problem, return_tensors="pt").to(device)
print(inputs)

outputs = model.generate(**inputs, 
                         do_sample=False,
                         max_new_tokens=1,
                         force_words_ids=rating_token_ids,
                         num_beams=2,
                         remove_invalid_values=True,
                         return_dict_in_generate=True,
                         output_scores=True, 
                         )
print(outputs)

first_scores = outputs["scores"][0]
print(first_scores)
print(first_scores.shape)
first_probs = torch.nn.functional.softmax(first_scores, dim=-1)
print(first_probs[0])
print(sum(first_probs[0]))

{'input_ids': tensor([[    1,   319,   395, 29929, 29900,  3823,  6034, 29938, 13733,  2820,
           278,  3978,   297,   278,  6795, 29899, 13058,  3538,  5305,   338,
          7436,   304,   395, 29955,   718, 29871, 29906, 29875,  7449, 29871,
          1724,   338,   278,  9819,  4280,  1353, 29973]], device='cuda:5'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:5')}
BeamSearchDecoderOnlyOutput(sequences=tensor([[    1,   319,   395, 29929, 29900,  3823,  6034, 29938, 13733,  2820,
           278,  3978,   297,   278,  6795, 29899, 13058,  3538,  5305,   338,
          7436,   304,   395, 29955,   718, 29871, 29906, 29875,  7449, 29871,
          1724,   338,   278,  9819,  4280,  1353, 29973,    13]],
       device='cuda:5'), sequences_scores=tensor([-0.0062], device='cuda:5'), scores=(tensor([[-2.0333e+01, -2.0216e+01, -2.5555e+00,  ..., -2.0804e+01,
         

In [36]:
# slow
# for idx, prob in enumerate(first_probs[0]):
#     if prob > 1e-5:
#         print(f"token_id: {idx}, token: {tokenizer.decode([idx])}, prob: {prob}")

# fast
# non_zero_idxs = torch.gt(first_probs.view(-1),1e-1)
threshold = 0.1
mask = first_probs > threshold
non_zero_idxs = torch.nonzero(mask)
print(non_zero_idxs)
non_zero_vals = first_probs.view(-1)[non_zero_idxs]
print(non_zero_vals)

tensor([[    0,    13],
        [    0, 29871]], device='cuda:5')
tensor([[1.4780e-09, 7.8869e-01],
        [1.4780e-09, 1.0245e-01]], device='cuda:5')
